In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import pandas as pd
import json, os, requests
from api_key import api_key

In [2]:
# ---------------
# Starbucks Data
# ---------------

#read Starbucks csv
starbucks_data_path = os.path.join("Source_Files","Starbucks_US_2018_11_12.csv")
starbucks_data = pd.read_csv(starbucks_data_path)

#pull data from Starbucks csv
clean_data = starbucks_data[['city', 'state', 'zip_code']]

SB_states_data = pd.DataFrame(clean_data.groupby('state')['city'].count())
SB_states_data.rename(columns={'city':'Starbucks Stores'}, inplace=True)


In [3]:
# -------------------
# Dunkin Donuts Data
# -------------------


#read the Dunkin json file
with open('dunkinDonuts.json', 'r') as file:
    results = json.load(file)
    data = results['data']
    
# Create dataframe full of relevant information about Dunkin Donuts stores in the US
# First create empty list for State, Zip code, and whether they have almond milk
states,zips,almond = [],[],[]

#Go item by item in the list, each item is a dictionary
for i in data:
    states.append(i['state'])
    zips.append(i['postal'])
    
    if i['almond']=='Y':
        almond.append(1)
    else:
        almond.append(0)

ddDF = pd.DataFrame({
    'State':states,
    'Zip Code':zips,
    'Almond':almond
})

totalDunkin = len(data)

#Group the dataframe of individual stores into a summary by state
grpbyState = ddDF.groupby('State')

#empty dataframe for the final results
DDtotals = pd.DataFrame()

DDtotals['Dunkin Stores'] = grpbyState.count()['Zip Code']
DDtotals['Dunkins with Almond Milk'] = grpbyState.mean()['Almond']
#DDtotals['Dunkins with Almond Milk'] = pd.Series(["{0:.0f}%".format(val * 100) for val in DDtotals['Dunkins with Almond Milk']], index = DDtotals.index)

In [4]:

# ------------
# Income Data
# ------------

# Request data with API key
api_key = api_key
url = "https://api.stlouisfed.org/geofred/shapes/file?"
query_url = f"https://api.stlouisfed.org/geofred/series/data?series_id=WIPCPI&api_key={api_key}&file_type=json&date=2019-01-01"
response = requests.get(query_url)

# Display API call response
data = response.json()
data_2019 = data["meta"]["data"]["2019"]


# Store state names and personal income to list
state, income, state_abv = [], [], []

# Make a for loop to go through the url response to get each state's personal income
for i in data_2019:
    state.append(i["series_id"][0:2])
    income.append(float(i["value"]))

    # Display personal income per capital into dataframe
personal_income_df = pd.DataFrame({"State": state, "Income": income})
personal_income_df.head()


# ----------------
# Population Data
# ----------------

file = "Source_Files/population_by_state_2019.xls"
df = pd.read_excel(file)

population = df[["State", "Population"]]


# -------------------
# Merge Income + Pop
# -------------------

GDPandPOP = pd.merge(population, personal_income_df, how="outer")
GDPandPOP=GDPandPOP.set_index('State')


In [5]:
# ---------------------
# Merge All Dataframes
# ---------------------

# SB_states_data = index of state and one column of total Starbucks Stores in that state
# DDtotals = index of state, columns of total DD stores and % with almond milk
# GDPandPOP = index of state, columns of income and population

stores = pd.merge(SB_states_data,DDtotals, left_index=True, right_index=True)
finalDF = pd.merge(GDPandPOP, stores, left_index=True, right_index=True)

In [6]:
# Create Dataframe for analysis by income

bins = [30000, 40000, 50000, 60000, 70000, 80000, 90000]
group_labels = ["30k-40k","40k-50k","50k-60k","60k-70k","70k-80k","80k-90k"]
finalDF["Income Bracket"] = pd.cut(finalDF['Income'], bins, labels=group_labels)

stores_by_income = finalDF.groupby("Income Bracket").sum()
stores_by_income = stores_by_income[['Starbucks Stores', 'Dunkin Stores']]
stores_by_income['%Almond'] = finalDF.groupby('Income Bracket').mean()['Dunkins with Almond Milk']
#stores_by_income['%Almond'] = pd.Series(["{0:.0f}%".format(val * 100) for val in stores_by_income['%Almond']], index = stores_by_income.index)

stores_by_income

,Starbucks Stores,Dunkin Stores,%Almond
Income Bracket,,,
30k-40k,33,8,38%
40k-50k,3063,1271,50%
50k-60k,4873,3654,52%
60k-70k,3855,666,41%
70k-80k,1375,4073,83%
80k-90k,91,26,85%


In [8]:
# Create Dataframe for analysis by population

popBins = [0,5000,10000,15000,20000,25000,30000,35000,40000]
popLabels = ['0-5mil','5-10mil','10-15mil','15-20mil','20-25mil','25-30mil','30-35mil','35-40mil']

finalDF['Pop Bracket'] = pd.cut(finalDF['Population'],popBins,labels=popLabels)
byPopDF = finalDF.groupby('Pop Bracket').sum()[['Starbucks Stores','Dunkin Stores']]
byPopDF['%Almond'] = finalDF.groupby('Pop Bracket').mean()['Dunkins with Almond Milk']
#byPopDF['%Almond'] = pd.Series(["{0:.0f}%".format(val * 100) for val in byPopDF['%Almond']], index = byPopDF.index)

byPopDF

,Starbucks Stores,Dunkin Stores,%Almond
Pop Bracket,,,
0-5mil,1782,1513,55%
5-10mil,3781,3266,57%
10-15mil,2191,2125,37%
15-20mil,679,1501,84%
20-25mil,741,1002,20%
25-30mil,1148,175,54%
30-35mil,0,0,nan%
35-40mil,2968,116,22%


In [9]:
finalDF

,Population,Income,Starbucks Stores,Dunkin Stores,Dunkins with Almond Milk,Income Bracket,Pop Bracket
AL,4907.965,44102.0,88,41,0.536585,40k-50k,0-5mil
AK,733.603,62629.0,51,2,0.000000,60k-70k,0-5mil
AZ,7291.843,45975.0,513,96,0.708333,40k-50k,5-10mil
AR,3020.985,44582.0,61,8,0.750000,40k-50k,0-5mil
CA,39437.610,66745.0,2968,116,0.224138,60k-70k,35-40mil
CO,5758.486,61159.0,501,45,0.555556,60k-70k,5-10mil
CT,3566.022,77273.0,129,524,0.912214,70k-80k,0-5mil
DE,976.668,54323.0,32,67,0.119403,50k-60k,0-5mil
DC,708.253,83111.0,91,26,0.846154,80k-90k,0-5mil
FL,21492.056,52391.0,741,1002,0.202595,50k-60k,20-25mil
